In [80]:
import requests
import numpy as np
import pandas as pd
import random
from datetime import datetime
from bs4 import BeautifulSoup as soup
import pyodbc

In [81]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [112]:
#Establishing connection with the database
pyodbc.drivers()
conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.34.15;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
cursor = conn.cursor()

In [104]:
#Reading all the necessary views in a pandas dataframe

case_master = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_V].[Case_Master_V]", conn)
case_entity = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_V].[Case_Entity_v]", conn)
case_entity_account = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_V].[Case_Entity_Accounts_V]", conn)
case_entity_report = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_V].[Case_Entity_Reports_V]", conn)
case_entity_location = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_V].[Case_Entity_Location_v]", conn)
case_sum = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore].[Case_Summary]", conn)


In [28]:
case_master.head()

,Case_Master_Id,Batch_Id,Report_Id,Re_Id,Fiu_Re_Id,Re_Name,Re_Type,Str_Report_Creation_Date,Version,Original_Batch_Id,Original_Report_Id,Fingate_Migrated_Data,Report_Type,Reporting_Period,Case_Master_Status_Id,Case_Priority,Is_Active_Flag,Case_Type_Flag,Fiu_Analyst,Case_System_Id,Create_Date,Create_By,Update_Date,Update_By,Create_Job_Id,Update_Job_Id
0,BA00000001,2022082210145134,69612,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,18,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
1,BA00000002,2022082210145134,69613,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,19,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
2,BA00000003,2022082210145134,69614,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,20,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
3,BA00000004,2022082210145134,69615,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,21,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
4,BA00000005,2022082210145134,69616,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,22,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111


In [29]:
case_entity.head()

,Case_Master_Id,Entity_Master_Id,Entity_Type,Entity_Name,Customer_Risk_Level,Pan,Passport_Number,Gstin,Unique_Company_Id,Date_Of_Incorporation,Date_Of_Birth,Gender,Ckyc_Number,Date_Of_Last_Ckyc,Is_Main_Entity_Flag,Degree_Of_Relationship,Source_Linkage,Is_Active_Flag,System_Case_Entity_Flag,Create_Date,Create_By,Update_Date,Update_By,Create_Job_Id,Update_Job_Id
0,BA00000005,1,O,Gaurav,None,None,None,None,None,NaT,NaT,None,None,None,1,0,Re-Provided,1,Y,2022-08-24 12:42:30.9633333,FIU-IND,2022-08-24 12:42:30.9633333,FIU-IND,1100,1111
1,BA00000007,2,O,Priyesh,None,None,None,None,None,NaT,NaT,None,None,None,1,0,Re-Provided,1,Y,2022-08-24 12:42:30.9633333,FIU-IND,2022-08-24 12:42:30.9633333,FIU-IND,1100,1111
2,BA00000011,3,O,Nitin,None,None,None,None,None,NaT,NaT,None,None,None,1,0,Re-Provided,1,Y,2022-08-24 12:42:30.9633333,FIU-IND,2022-08-24 12:42:30.9633333,FIU-IND,1100,1111
3,BA00000001,4,I,Vaji Singh,High,AENPC7777H,A11111111,None,None,NaT,1998-01-01,Male,10000000000000,2019-06-20 00:00:00.0000000,1,0,Re-Provided,1,Y,2022-08-24 12:42:30.9633333,FIU-IND,2022-08-24 12:42:30.9633333,FIU-IND,1100,1111
4,BA00000002,4,I,Shyam Singh,High,AGLPR9772P,A11111112,None,None,NaT,1980-02-01,Male,10000000000000,2019-06-20 00:00:00.0000000,1,0,Re-Provided,1,Y,2022-08-24 12:42:30.9633333,FIU-IND,2022-08-24 12:42:30.9633333,FIU-IND,1100,1111


In [30]:
case_entity_account.head()

,Case_Master_Id,Entity_Master_Id,Case_Entity_Accounts_Id,Account_Number,Branch_Code,Ifsc_Code,Account_Type,Create_Date,Create_By,Update_Date,Update_By,Create_Job_Id,Update_Job_Id,Re_Id,Re_Name,Re_Branch_Address,Re_Branch_Phone,Bank_Name,Source_Linkage
0,BA00000001,4,1,13967946867,198322,DQRI8899I,Current Account,2022-08-24 12:51:21.4700000,FIU-IND,2022-08-24 12:51:21.4700000,FIU-IND,1100,1111,14901,Test DQR PU,", , , , , ,",None,None,Re-Provided
1,BA00000002,4,2,13967946867,198322,DQRI8899I,Current Account,2022-08-24 12:51:21.4700000,FIU-IND,2022-08-24 12:51:21.4700000,FIU-IND,1100,1111,14901,Test DQR PU,", , , , , ,",None,None,Re-Provided
2,BA00000003,4,3,13967946867,198322,DQRI8899I,Current Account,2022-08-24 12:51:21.4700000,FIU-IND,2022-08-24 12:51:21.4700000,FIU-IND,1100,1111,14901,Test DQR PU,", , , , , ,",None,None,Re-Provided
3,BA00000004,4,4,13967946867,198322,DQRI8899I,Current Account,2022-08-24 12:51:21.4700000,FIU-IND,2022-08-24 12:51:21.4700000,FIU-IND,1100,1111,14901,Test DQR PU,", , , , , ,",None,None,Re-Provided
4,BA00000001,4,5,6558260792,198324,DQRI8899I,Current Account,2022-08-24 12:51:21.4700000,FIU-IND,2022-08-24 12:51:21.4700000,FIU-IND,1100,1111,14901,Test DQR PU,", , , , , ,",None,None,Re-Provided


In [31]:
case_entity_report

,Case_Master_Id,Entity_Master_Id,Batch_Id,Report_Id,Re_Id,Account_Number,Transaction_Id,Related_Report_Case_Id,Re_Name,Version,Original_Batch_Id,Original_Report_Id,Fingate_Migrated_Data,Reporting_Period,Report_Date,Report_Type,Report_Type_Id,Transaction_Amount,Transaction_Type,Transaction_Mode,Is_Active_Flag,Create_Date,Create_By,Update_Date,Update_By,Create_Job_Id,Update_Job_Id
0,BA00000005,1,2022082210145134,69616,14901,8788822415,345237567866,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,9.03478e+007,Deposit,,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
1,BA00000011,3,2022082210145134,69622,14901,9873479356,45254421,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,2.37052e+007,Deposit,Cash,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
2,BA00000001,4,2022082210145134,69612,14901,13967946868,541236521234545,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,1.02e+007,Deposit,,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
3,BA00000002,4,2022082210145134,69613,14901,13967946867,43136236712454,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,2.905e+007,Deposit,,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
4,BA00000003,4,2022082210145134,69614,14901,5235434534,85100000000444,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,1.8875e+007,Deposit,,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
5,BA00000004,4,2022082210145134,69615,14901,6558260792,876561267511,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,6.53728e+007,Deposit,,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
6,BA00000009,7,2022082210145134,69620,14901,4588822415,98705351,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,7.93126e+007,Deposit,Cash,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111
7,BA00000010,8,2022082210145134,69621,14901,6588260792,76515241,None,Test DQR PU,1,None,None,N,2022-8-22,2022-08-23 12:33:31.5533333,STR,101,7.61573e+007,Deposit,Cash,1,2022-08-24 12:42:31.0666667,Sourav_ETL,2022-08-24 12:42:31.0666667,Sourav_ETL,1100,1111


In [90]:
case_master

,Case_Master_Id,Batch_Id,Report_Id,Re_Id,Fiu_Re_Id,Re_Name,Re_Type,Str_Report_Creation_Date,Version,Original_Batch_Id,Original_Report_Id,Fingate_Migrated_Data,Report_Type,Reporting_Period,Case_Master_Status_Id,Case_Priority,Is_Active_Flag,Case_Type_Flag,Fiu_Analyst,Case_System_Id,Create_Date,Create_By,Update_Date,Update_By,Create_Job_Id,Update_Job_Id
0,BA00000001,2022082210145134,69612,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,18,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
1,BA00000002,2022082210145134,69613,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,19,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
2,BA00000003,2022082210145134,69614,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,20,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
3,BA00000004,2022082210145134,69615,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,21,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
4,BA00000005,2022082210145134,69616,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,22,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
5,BA00000006,2022082210145134,69617,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,23,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
6,BA00000007,2022082210145134,69618,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,24,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
7,BA00000008,2022082210145134,69619,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,25,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
8,BA00000009,2022082210145134,69620,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,26,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111
9,BA00000010,2022082210145134,69621,14901,BA00000255,Test DQR PU,BA,2022-08-23 12:33:31.553,1,None,None,N,STR,2022-8-22,1,None,1,1,testfiuanalyst@gm.co,27,2022-08-26 13:20:48.3766667,FIU-IND,2022-08-26 13:20:48.3766667,FIU_IND,1100,1111


In [92]:
case_master[case_master['Case_Type_Flag'] == 1]['Case_Master_Id'].to_list()

['BA00000001',
 'BA00000002',
 'BA00000003',
 'BA00000004',
 'BA00000005',
 'BA00000006',
 'BA00000007',
 'BA00000008',
 'BA00000009',
 'BA00000010',
 'BA00000011',
 'BA00000012',
 'BA00000013',
 'BA00000014',
 'BA00000015',
 'BA00000016',
 'BA00000017']

In [105]:
#Reading all the unique case master IDs stored in the case_master dataframe

cm_id = list(set(case_master[case_master['Case_Type_Flag'] == 1]['Case_Master_Id'].to_list()) - set(case_sum['Case_Master_Id'].to_list()))
#making a list to iterate
# cm_id = list(cm_id) 


In [106]:
#Dictionary that maps geder with pronouns
gen_dict = {'Male':'His','Female':'Her'}

#creating a dataframe to store the case summary 
case_data = pd.DataFrame()
#list to store the case summary of each case
summary_list = []

for cmid in cm_id:
    #Creating an HTML file to store the HTML
    html_file = open(r"C:\Users\FIU\Desktop\case_summary\cs_"+str(cmid)+".html",'w')
    css = '''<!DOCTYPE html>
        <html>
        <head>
        <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
        <style>
                ul{
                    list-style-type:disc;
                }
                body{
                font-family:"Calibri, sans-serif";
                word-wrap:break-word;
                margin-left: 20px;
                }
            </style>
        </head>
        <body lang="EN-US">
        <div>
        '''
    #Writing the CSS in the html file
    html_file.write(css)
    #beginign of unordered list
    html_file.write("<ul>")
    print(cmid)
    #counting the number of main entities
    num_ME = len(case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)])+ len(case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 'Y')])
    #Counting the numbe of other entties
    other_E = len(case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 0)])+ len(case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 'N')])
    #Creating a list of names of Main entites for person
    ME_list_per = case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]['Entity_Name'].to_list()
    #Creating a list of names of Main entites for person
    ME_list_org = case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]['Entity_Name'].to_list()
    #Main entity person master ids dataframe
    me_per_id = case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]
    #Main entity organization master ids dataframe
    me_org_id = case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]
    #list of person master IDs
    me_per_id_list = me_per_id['Entity_Master_Id'].to_list()
    
    #This data is currently unavailable
    num_high_conf_ent = 10
    high_risk_high_conf = 5
    top_three_high_risk_high_conf = ['Kuldeep','Karamveer','Saurabh']
    
    main_str_num_loc_count = 0 #main str number of location count
    main_str_credit_count = 0 
    main_str_debit_count = 0
    main_str_cum_credit = 0 #main str cumulative credit
    main_str_cum_debit = 0
    
    #Number of past cases
    num_past_case = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid)]['Related_Report_Case_Id'].unique())
    #Count of various report
    str_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'STR')]['Report_Id'].unique())
    ctr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'CTR')]['Report_Id'].unique())
    ntr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'NTR')]['Report_Id'].unique())
    ptr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'PTR')]['Report_Id'].unique())
    cbwtr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'CBWTR')]['Report_Id'].unique())
                                       
    str_main_loc = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'STR')]
    
    #Credit and Debit count of various reports
    str_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Id'].unique())
    str_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Id'].unique())

    ctr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Id'].unique())
    ctr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Id'].unique())

    ntr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Id'].unique())
    ntr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Id'].unique())

    ptr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Id'].unique())
    ptr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Id'].unique())

    cbwtr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Id'].unique())
    cbwtr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Id'].unique())

    str_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Amount'].sum()
    str_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Amount'].sum()

    ctr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Amount'].sum()
    ctr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Amount'].sum()

    ntr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Amount'].sum()
    ntr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Amount'].sum()

    ptr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Amount'].sum()
    ptr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Amount'].sum()

    cbwtr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Amount'].sum()
    cbwtr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Amount'].sum()
    
    #Begining of case smmary with a list
    s = "<li>This case is a <b>"+str(8)+"</b> risk case.</li>\n"

    if num_ME == 1:
        s = s+"<li>There is <b>"+str(num_ME)+"</b> main entity reported in the case ,"
    else:
        s = s+"<li>There are <b>"+str(num_ME)+"</b> main entities reported in the case ,"
        
    for i in range(len(ME_list_per)):
        if i == len(ME_list_per)-1:
            s=s+"<b>"+ME_list_per[i]+".</b>"
        else:
            s = s+"<b>"+ME_list_per[i]+",</b>"
            
    for i in range(len(ME_list_org)):
        if i == len(ME_list_org)-1:
            s=s+"<b>"+ME_list_org[i]+".</b>"
        else:
            s = s+"<b>"+ME_list_org[i]+",</b>"
    s = s+"<ol>" #Begining of the ordered list
    
    #Iterating over the Main Entity for person dataframe and embedding the fields
    for ind,row in me_per_id.iterrows():
        
        name = row['Entity_Name']
        gender = row['Gender']
        age = 0 #row['Age']
        risk = row['Customer_Risk_Level']
        occ = 'Blank' #row['occupationId'] #Not available currently
        num_acc = len(case_entity_account[(case_entity_account['Entity_Master_Id'] == row['Entity_Master_Id'])]['Account_Number'].unique())
        name_loc = case_entity_location[(case_entity_location['Addresstype'] == 'Branch' ) & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique()
        num_loc = len(case_entity_location[(case_entity_location['Addresstype'] == 'Branch') & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique())
        ann_credit = case_entity_report[(case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_debit = case_entity_report[(case_entity_report['Transaction_Type'] == 'debit') &(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_cash_cred = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        ann_cash_debit = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        main_str_num_loc_count += num_loc
        main_str_credit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'credit')])
        main_str_debit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'debit')])
        main_str_cum_credit += ann_credit
        main_str_cum_debit += ann_debit
        print(name,gender,age,risk)
        print(num_acc,name_loc,ann_credit,ann_debit)
        print(ann_cash_cred,ann_cash_debit)
        s = s+"<li><b>"+name+"</b> is <b>37</b> year old <b>"+gender+"</b> and is a <b>"+str(risk)+"</b> risk individual. <b>"+gen_dict[gender]+"</b> occupation is <b>"+occ+"</b>"
        s = s+"<ul><li><b>"+name+"</b> has <b>"+str(num_acc)+"</b> accounts across <b>"+str(num_loc)+"</b> Locations-<b> "+ str(name_loc) +"</b></li></ul>"
        s = s+"<ul><li><b>"+name+"</b> has conducted annual transactions totaling INR <b>"+str(ann_credit)+"</b> (Credit) and <b>"+str(ann_debit)+"</b> (Debit) out of which INR <b>"+str(ann_cash_cred)+"</b> (Cash credit) and INR <b>"+str(ann_cash_debit)+"</b> (Cash Debit) has been done.</li></ul>"

    #Iterating over the Main Entity for organization dataframe and embedding the fields
    for ind,row in me_org_id.iterrows():
        name = row['Entity_Name']
        doi = row['Date_Of_Incorporation']
        risk = row['Customer_Risk_Level']
        occ = 'Blank' #row['occupationId'] #Not available currently
        num_acc = len(case_entity_account[(case_entity_account['Entity_Master_Id'] == row['Entity_Master_Id'])]['Account_Number'].unique())
        name_loc = case_entity_location[(case_entity_location['Addresstype'] == 'Branch' ) & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique()
        num_loc = len(case_entity_location[(case_entity_location['Addresstype'] == 'Branch') & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique())
        ann_credit = case_entity_report[(case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_debit = case_entity_report[(case_entity_report['Transaction_Type'] == 'debit') &(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_cash_cred = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        ann_cash_debit = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        main_str_num_loc_count += num_loc
        main_str_credit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'credit')])
        main_str_debit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'debit')])
        main_str_cum_credit += ann_credit
        main_str_cum_debit += ann_debit
        print(name,doi,occ,risk)
        s = s+ name+ " has been established on "+ str(doi) + " with a "+ occ+ " Type of Business and is a "+ str(risk) +" risk organization." 
        #Further nesting the unordered list in an Ordered list
        s = s+"<ul><li><b>"+name+"</b> has <b>"+str(num_acc)+"</b> accounts across <b>"+str(num_loc)+"</b> Locations-<b> "+ str(name_loc) +"</b></li></ul>"
        s = s+"<ul><li><b>"+name+"</b> has conducted annual transactions totaling INR <b>"+str(ann_credit)+"</b> (Credit) and <b>"+str(ann_debit)+"</b> (Debit) out of which INR <b>"+str(ann_cash_cred)+"</b> (Cash credit) and INR <b>"+str(ann_cash_debit)+"</b> (Cash Debit) has been done.</li></ul>"
    
    s = s+"</ol></li>"    
    html_file.write(s)
    ul = "<li>There are other<b> "+str(other_E) +"</b> reported in the STR out of which <b> "+str(0)+"</b> entities are high risk.</li>"
    ul += "<li>"+str(num_high_conf_ent)+" high confidence entities are also linked to the case out of which "+str(high_risk_high_conf) +" entities are high risk."
    for i in range(len(top_three_high_risk_high_conf)):
        if i == len(top_three_high_risk_high_conf)-1:
            ul+= "<b>"+top_three_high_risk_high_conf[i]+"</b>"
        else:
            ul+= "<b>"+top_three_high_risk_high_conf[i]+",</b>"
    ul += " are the top 3 high risk linked entities of the case.</li>"
    ul += "<li>The case entities have a total of past "+str(num_past_case)+" cases against them."
    ul += "<li>The Main STR has "+str(main_str_num_loc_count) +" main transaction locations and the transaction counts involved are "+str(main_str_credit_count)+" (Credit) and "+str(main_str_debit_count)+" (Debit). The total amount in the STR is reported  INR "+str(main_str_cum_credit)+" (credit) and INR "+str(main_str_cum_debit) +"(Debit) </li>"
    ul += "<li>The linked reports include "+str(str_count)+" STRs, "+str(ctr_count)+" CTRs, "+str(ntr_count)+" NTRs, and "+str(ptr_count)+" PTRs.</li>" 
    ul += "<ul><li>The linked STR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(str_count_credit)+" (Credit) and "+str(str_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(str_gross_credit)+" (credit) and INR "+ str(str_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked CTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(ctr_count_credit)+" (Credit) and "+str(ctr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(ctr_gross_credit)+" (credit) and INR "+ str(ctr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked NTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(ntr_count_credit)+" (Credit) and "+str(ntr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(ntr_gross_credit)+" (credit) and INR "+ str(ntr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked PTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(ptr_count_credit)+" (Credit) and "+str(ptr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(ptr_gross_credit)+" (credit) and INR "+ str(ptr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked CBWTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(cbwtr_count_credit)+" (Credit) and "+str(cbwtr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(cbwtr_gross_credit)+" (credit) and INR "+ str(cbwtr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details

    
    
    html_file.write(ul)
    html_file.write("</ul>")
    end = '''</div>
            </body>
            </html>'''
    #saving the file
    html_file.write(end)
    #Closing the html file
    html_file.close()
    
    #Reading the content of the HTML file
    with open(r"C:\Users\FIU\Desktop\case_summary\cs_"+str(cmid)+".html",'r') as fo:
        data = fo.read()
        #Appeding the data in the summary_list list
        summary_list.append(data)
        

    
#     break
# case_data['Case_Master_Id'] = cm_id
# case_data['Case_Summary'] = summary_list

BA00000011
Nitin NaT Blank None
BA00000012
Khusbu singh Male 0 None
1 [] 0 0
0 0
BA00000002
Shyam Singh Male 0 High
4 [] 0 0
0 0
BA00000001
Vaji Singh Male 0 High
4 [] 0 0
0 0
BA00000005
Gaurav NaT Blank None
BA00000008
ZOLO LIMITED NaT Blank None
BA00000017
divyansh 2018-07-15 00:00:00 Blank Medium
BA00000013
Hemant NaT Blank None
BA00000003
Suresh Kumar  Male 0 High
4 [] 0 0
0 0
BA00000015
aman prasad Male 0 None
1 [] 0 0
0 0
BA00000006
POPANI  PRIVATE LIMITED NaT Blank None
BA00000014
Sahna kumar Male 0 None
1 [] 0 0
0 0
BA00000007
Priyesh NaT Blank None
BA00000004
Akash Rout Male 0 High
4 [] 0 0
0 0
BA00000016
rupesh 2015-09-13 00:00:00 Blank Medium
BA00000010
yamraj enterprises 2010-09-20 00:00:00 Blank Medium
BA00000009
zee telecom 2009-08-22 00:00:00 Blank Medium


In [107]:
#Reading the case_master_id for adhoc cases
# cm_id_ad = case_master[case_master['Case_Type_Flag'] == 2]['Case_Master_Id'].to_list()
cm_id_ad = list(set(case_master[case_master['Case_Type_Flag'] == 2]['Case_Master_Id'].to_list()) - set(case_sum['Case_Master_Id'].to_list()))

In [108]:
# summary_adhoc_list = []
for cmid in cm_id_ad:
    
    html_file = open(r"C:\Users\FIU\Desktop\case_summary\cs_adhoc_"+str(cmid)+".html",'w')
    css = '''<!DOCTYPE html>
        <html>
        <head>
        <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
        <style>
                ul{
                    list-style-type:disc;
                }
                body{
                font-family:"Calibri, sans-serif";
                word-wrap:break-word;
                margin-left: 20px;
                }
            </style>
        </head>
        <body lang="EN-US">
        <div>
        '''
    html_file.write(css)
    html_file.write("<ul>")
    print(cmid)
    num_ME = len(case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)])+ len(case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 'Y')])
    other_E = len(case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 0)])+ len(case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 'N')])
    ME_list_per = case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]['Entity_Name'].to_list()
    ME_list_org = case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]['Entity_Name'].to_list()
    me_per_id = case_entity[(case_entity['Entity_Type'] == 'I') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]
    me_org_id = case_entity[(case_entity['Entity_Type'] == 'O') & (case_entity['Case_Master_Id'] == cmid) & (case_entity['Is_Main_Entity_Flag'] == 1)]
    me_per_id_list = me_per_id['Entity_Master_Id'].to_list()
    
    main_str_num_loc_count = 0
    main_str_credit_count = 0
    main_str_debit_count = 0
    main_str_cum_credit = 0
    main_str_cum_debit = 0
    
    #This data is not available for now(conf)
    num_high_conf_ent = 10
    high_risk_high_conf = 4
    
    num_past_case = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid)]['Related_Report_Case_Id'].unique())
    str_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'STR')]['Report_Id'].unique())
    ctr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'CTR')]['Report_Id'].unique())
    ntr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'NTR')]['Report_Id'].unique())
    ptr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'PTR')]['Report_Id'].unique())
    cbwtr_count = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'CBWTR')]['Report_Id'].unique())
                                       
    str_main_loc = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Report_Type'] == 'STR')]
    
    str_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Id'].unique())
    str_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Id'].unique())

    ctr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Id'].unique())
    ctr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Id'].unique())

    ntr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Id'].unique())
    ntr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Id'].unique())

    ptr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Id'].unique())
    ptr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Id'].unique())

    cbwtr_count_credit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Id'].unique())
    cbwtr_count_debit = len(case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Id'].unique())

    str_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Amount'].sum()
    str_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'STR')]['Transaction_Amount'].sum()

    ctr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Amount'].sum()
    ctr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CTR')]['Transaction_Amount'].sum()

    ntr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Amount'].sum()
    ntr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'NTR')]['Transaction_Amount'].sum()

    ptr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Amount'].sum()
    ptr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'PTR')]['Transaction_Amount'].sum()

    cbwtr_gross_credit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Amount'].sum()
    cbwtr_gross_debit = case_entity_report[(case_entity_report['Case_Master_Id'] == cmid) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Report_Type'] == 'CBWTR')]['Transaction_Amount'].sum()
    
    
    s = "<li>This case is a <b>"+str(8)+"</b> risk case.</li>\n"
    

    if num_ME == 1:
        s = s+"<li>There is <b>"+str(num_ME)+"</b> main entity selected in the case,"
    else:
        s = s+"<li>There are <b>"+str(num_ME)+"</b> main entities reported in the case ,"
        
    for i in range(len(ME_list_per)):
        if i == len(ME_list_per)-1:
            s=s+"<b>"+ME_list_per[i]+".</b>"
        else:
            s = s+"<b>"+ME_list_per[i]+",</b>"
            
    for i in range(len(ME_list_org)):
        if i == len(ME_list_org)-1:
            s=s+"<b>"+ME_list_org[i]+".</b>"
        else:
            s = s+"<b>"+ME_list_org[i]+",</b>"
    s = s+"<ol>"
    
    
    for ind,row in me_per_id.iterrows():
        
        name = row['Entity_Name']
        gender = row['Gender']
        age = 0 #row['Age']
        risk = row['Customer_Risk_Level']
        occ = 'Blank' #row['occupationId'] #Not available currently
        num_acc = len(case_entity_account[(case_entity_account['Entity_Master_Id'] == row['Entity_Master_Id'])]['Account_Number'].unique())
        name_loc = case_entity_location[(case_entity_location['Addresstype'] == 'Branch' ) & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique()
        num_loc = len(case_entity_location[(case_entity_location['Addresstype'] == 'Branch') & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique())
        ann_credit = case_entity_report[(case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_debit = case_entity_report[(case_entity_report['Transaction_Type'] == 'debit') &(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_cash_cred = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        ann_cash_debit = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        main_str_num_loc_count += num_loc
        main_str_credit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'credit')])
        main_str_debit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'debit')])
        main_str_cum_credit += ann_credit
        main_str_cum_debit += ann_debit
        print(name,gender,age,risk)
        print(num_acc,name_loc,ann_credit,ann_debit)
        print(ann_cash_cred,ann_cash_debit)
        s = s+"<li><b>"+name+"</b> is <b>37</b> year old <b>"+gender+"</b> and is a <b>"+str(risk)+"</b> risk individual. <b>"+gen_dict[gender]+"</b> occupation is <b>"+occ+"</b>"
        s = s+"<ul><li><b>"+name+"</b> has <b>"+str(num_acc)+"</b> accounts across <b>"+str(num_loc)+"</b> Locations-<b> "+ str(name_loc) +"</b></li></ul>"
       

        
    for ind,row in me_org_id.iterrows():
        name = row['Entity_Name']
        doi = row['Date_Of_Incorporation']
        risk = row['Customer_Risk_Level']
        occ = 'Blank' #row['occupationId']
        num_acc = len(case_entity_account[(case_entity_account['Entity_Master_Id'] == row['Entity_Master_Id'])]['Account_Number'].unique())
        name_loc = case_entity_location[(case_entity_location['Addresstype'] == 'Branch' ) & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique()
        num_loc = len(case_entity_location[(case_entity_location['Addresstype'] == 'Branch') & (case_entity_location['Entity_Master_Id'] == row['Entity_Master_Id'])]['District'].unique())
        ann_credit = case_entity_report[(case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_debit = case_entity_report[(case_entity_report['Transaction_Type'] == 'debit') &(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id'])]['Transaction_Amount'].sum()
        ann_cash_cred = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'credit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        ann_cash_debit = case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Type'] == 'debit') & (case_entity_report['Transaction_Mode'] == 'cash')]['Transaction_Amount'].sum()
        main_str_num_loc_count += num_loc
        main_str_credit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'credit')])
        main_str_debit_count += len(case_entity_report[(case_entity_report['Entity_Master_Id'] == row['Entity_Master_Id']) & (case_entity_report['Transaction_Mode'] == 'debit')])
        main_str_cum_credit += ann_credit
        main_str_cum_debit += ann_debit

        s = s+ name+ " has been established on "+ doi + " with a "+ occ+ " Type of Business and is a "+ str(risk) +" risk organization." 
        s = s+"<ul><li><b>"+name+"</b> has <b>"+str(num_acc)+"</b> accounts across <b>"+str(num_loc)+"</b> Locations-<b> "+ str(name_loc) +"</b></li></ul>"
        
    
    s = s+"</ol></li>"    
    html_file.write(s)
    ul = "<li>"+str(num_high_conf_ent)+" high confidence entities are also linked to the case out of which "+str(high_risk_high_conf) +" entities are high risk."
    ul += "<li>The case entities have a total of past "+str(num_past_case)+" cases against them."    
    ul += "<li>The linked reports include "+str(str_count)+" STRs, "+str(ctr_count)+" CTRs, "+str(ntr_count)+" NTRs, and "+str(ptr_count)+" PTRs.</li>" 
    ul += "<ul><li>The linked STR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(str_count_credit)+" (Credit) and "+str(str_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(str_gross_credit)+" (credit) and INR "+ str(str_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked CTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(ctr_count_credit)+" (Credit) and "+str(ctr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(ctr_gross_credit)+" (credit) and INR "+ str(ctr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked NTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(ntr_count_credit)+" (Credit) and "+str(ntr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(ntr_gross_credit)+" (credit) and INR "+ str(ntr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked PTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(ptr_count_credit)+" (Credit) and "+str(ptr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(ptr_gross_credit)+" (credit) and INR "+ str(ptr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details
    ul += "<ul><li>The linked CBWTR has/have  "+ str(5) +" main transaction locations and the transaction counts involved are  "+str(cbwtr_count_credit)+" (Credit) and "+str(cbwtr_count_debit)+ " (Debit). The total amount in these reports is reported  INR "+str(cbwtr_gross_credit)+" (credit) and INR "+ str(cbwtr_gross_debit)+" (Debit).</li></ul>" #Doing this to display Report details

    
    
    html_file.write(ul)
    html_file.write("</ul>")
    end = '''</div>
            </body>
            </html>'''
    html_file.write(end)
    html_file.close()

    with open(r"C:\Users\FIU\Desktop\case_summary\cs_"+str(cmid)+".html",'r') as fo:
        data = fo.read()
        summary_list.append(data)

#Here Appedning the case_master_Ids for the adhoc cases as well as non-adhoc cases
cm_id.extend(cm_id_ad)
#Creating two new columns in the case_data df and assinging a list of case_master_Ids and 
#case_summaries stored in the summary_list
case_data['Case_Master_Id'] = cm_id
case_data['Case_Summary'] = summary_list

In [116]:
case_data

,Case_Master_Id,Case_Summary
0,BA00000011,<!DOCTYPE html>\n <html>\n <head...
1,BA00000012,<!DOCTYPE html>\n <html>\n <head...
2,BA00000002,<!DOCTYPE html>\n <html>\n <head...
3,BA00000001,<!DOCTYPE html>\n <html>\n <head...
4,BA00000005,<!DOCTYPE html>\n <html>\n <head...
5,BA00000008,<!DOCTYPE html>\n <html>\n <head...
6,BA00000017,<!DOCTYPE html>\n <html>\n <head...
7,BA00000013,<!DOCTYPE html>\n <html>\n <head...
8,BA00000003,<!DOCTYPE html>\n <html>\n <head...
9,BA00000015,<!DOCTYPE html>\n <html>\n <head...


In [122]:
#Establishing a connection wih the database and writing the content of the case_data dataframe in the case_summary_V table
pyodbc.drivers()
conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.34.15;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_App_USR;PWD=F!NCore@123;TDS_Version=7.2;')
cursor = conn.cursor()
for ind,row in case_data.iterrows():
#     print(row)
    cursor.execute("""insert into [FINCORE_DB].[Fincore_V].[Case_Summary_V] (Case_Master_Id,Case_Summary,Create_Job_Id,Update_Job_Id)
                values(?,?,?,?)""",(row['Case_Master_Id'],row['Case_Summary'],'100','100'))
    
conn.commit()
conn.close()